In [1]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

# Laser Shading By adjusting focus on the Z-axis

# Code:

In [2]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # [mm]
BlockLength = 2 * BlockHeight  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))

89.0 x 178.0 x 19.0


In [3]:
def init(**kwargs):
    program = GCode.GCode(**kwargs)
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    program.M5()
    return program


def end(**kwargs):
    program = GCode.GCode(**kwargs)
    program.M5()
    return program


In [4]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])


In [5]:
cnc.reset(home=True)

(3, ['', "Grbl 1.1f ['$' for help]", "[MSG:'$H'|'$X' to unlock]"])

In [6]:
if "Alarm" in cnc.status:
    print("Homing.")
    cnc.home()


Homing.


# Generate Program

Generate the G-code program that will test how the laser focus affects shading.

- Laser 'focused' at -14mm.
- Testing 17 positions between [-28, 0]
- Testing 3 different feed rates [100, 200, 300]
- Testing 5 different power settings between [100, 255]

In [ ]:
# Define a gap between horizontal lines
vgap = 5 # [mm]
# Generate a list of horizontal line vertical coordinates
vs = np.arange( vgap, BlockHeight, vgap)
vs


In [ ]:
# Define a horizontal distance to test lazerin.
hgap = 10 # [mm]
hs = np.arange(0, BlockLength, hgap)
hs


In [ ]:
# Total travel range, starts at ztravel, ends at 0. "negative space"(?)
ztravel = -28
# Generate a number of z-axis test positions.
# Each z-height will have 10 mm marked at the given feed rate and power.
zs = np.linspace(start=ztravel, stop=0, num=len(hs) - 1, endpoint=True)
zs


In [ ]:
# Laser powers to test.
powers = np.linspace(100, 255, 5, endpoint=True, dtype=np.uint8)
powers


In [ ]:
program = GCode.GCode(machine=cnc)
program.G0(F=500)
program.G1(F=300)
program.M3(S=1)

feeds = [100, 200, 300]

for idx1, feed in enumerate(feeds):  # [mm/min]
    program.G1(F=feed)
    for idx2, power in enumerate(powers):
        program.M4(S=power)
        program.M5()
        
        v_idx = idx1 * len(feeds) + idx2
        Y = vs[v_idx]
        program.G0(X=hs[0], Y=Y)
        for idx3, z in enumerate(zs):
            x0 = hs[idx3]
            xf = hs[idx3 + 1]
            program.G0(Z=z, X=x0)
            program.M4(S=power)
            program.G1(X=xf, Y=Y, Z=z, F=feed)
            program.M5() # Laser. Off.
            break
        
        program.M5() # Laser. Off.
        break
        
    program.M5() # Laser. Off.
    break

In [ ]:
cnc.init()

In [ ]:
program.run()

In [ ]:
cnc.reset()

In [ ]:
cnc.status

In [ ]:
cnc.home()

In [ ]:
init(machine=cnc).run()

In [ ]:
cnc.cmd("$C")

In [ ]:
results=list()
for l in program.buffer:
    results.append(cnc.cmd(l))

In [ ]:
results

In [ ]:
cnc.run(program)

In [ ]:
cnc.status

In [ ]:
cnc.cmd("$C")

In [ ]:
import time

In [ ]:
laser_mode = cnc.laser_mode
while laser_mode is None:
    cnc.reset()
    time.sleep(0.5)
    laser_mode = cnc.laser_mode
status = cnc.status
while "Alarm" in status:
    cnc.home()
    
    status = cnc.status

In [ ]:
cnc.ho

In [ ]:
cnc.status